In [12]:
import json
import requests
import random
from ipyleaflet import (
    Map,
    TileLayer,
    FullScreenControl,
    ScaleControl,
    ZoomControl,
    Marker,
    SearchControl,
    AwesomeIcon,
    WidgetControl,
    GeoJSON,
    Popup,
)
from ipywidgets import IntSlider, ColorPicker, Button, link, HTML, jslink

m = Map(center=(47, 2), zoom=5, zoom_control=False)

# create Zoom_control
m.add_control(ZoomControl(position="topleft"))
# create Scale_Control
m.add_control(ScaleControl(position="bottomleft"))
# Create Full_Screen_Control
m.add_control(FullScreenControl())


zoom_slider = IntSlider(description="Zoom level:", min=0, max=15, value=7)
jslink((zoom_slider, "value"), (m, "zoom"))
widget_control1 = WidgetControl(widget=zoom_slider, position="topright")
m.add_control(widget_control1)


# Create a local Tile Layer concerning the irsn map
radon = TileLayer(
    url="https://cartoradon.irsn.fr/static/r/{z}/{x}/{y}.png", tms=True, opacity=0.5
)
m.add_layer(radon)

# Create a function get_data which fetch the datas from the irsn application
def get_data():
    r = requests.get(
        "https://cartoradon.irsn.fr/commune.py/communes/lonlat/{};{}".format(lng, lat)
    )
    return r.json()


# take a method on_interation providing Map object to obtain the coordinates
def on_interaction(event, type, coordinates):

    if type == "click":
        data = get_data(coordinates[1], coordinates[0])

        # Display geojson
        geometry = data["geometry"]
        geojson = GeoJSON(
            data=geometry,
            style={"opacity": 1, "dashArray": "9", "fillOpacity": 0.1, "weight": 1},
            hover_style={"color": "white", "dashArray": "0", "fillOpacity": 0.5},
        )
        m.add_layer(geojson)

        # Create Popup
        message1 = HTML()
        dict_properties = data["properties"]
        get_name = dict_properties.get("name")
        get_class = dict_properties.get("classr")
        message1.value = "Ville: {}, Classe : {}".format(get_name, get_class)
        popup = Popup(
            location=coordinates,
            child=message1,
            close_button=True,
            auto_close=True,
            close_on_escape_key=False,
        )
        m.add_layer(popup)


m.on_interaction(on_interaction)

m

Map(center=[47, 2], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Zoom…